In [1]:
import pandas as pd
import numpy as np
import requests
import string # special operations on strings
import spacy # language models
import nltk
import re
import gensim
import gensim.corpora as corpora
import pyLDAvis
import pyLDAvis.gensim_models
import warnings

from bs4 import BeautifulSoup as bs
from nltk import tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet,stopwords
from nltk import ngrams
from textblob import TextBlob, Word, Blobber
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud,STOPWORDS

%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
link1 = "https://thebridge.in" 
link2 = "https://www.bbc.com/sport"  
link3 = "https://indianexpress.com/section/sports/" 
link4 = "https://www.timesnownews.com/sports"

In [3]:
def getHTMLdocument(url):
    response = requests.get(url)
    return response.text

In [4]:
page1 = getHTMLdocument(link1)
page2 = getHTMLdocument(link2)
page3 = getHTMLdocument(link3)
page4 = getHTMLdocument(link4)

In [5]:
soup1 = bs(page1,'html.parser')
soup2 = bs(page2,'html.parser')
soup3 = bs(page3,'html.parser')
soup4 = bs(page4,'html.parser')

In [6]:
print(soup1.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Indian Sports Home - Latest News, Results, Stories | The Bridge
  </title>
  <link href="/images/ico/favicon.ico?v=1" rel="icon" type="image/x-icon"/>
  <link href="/images/ico/favicon.ico?v=1" rel="shortcut icon" type="image/x-icon"/>
  <meta charset="utf-8"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0,user-scalable=no" name="viewport"/>
  <meta content="Indian Sports Home - Latest News, Results, Stories | The Bridge" name="description"/>
  <meta content="The Bridge - Home of Indian Sports" name="application-name"/>
  <meta content="Indian Sports Home - Latest News, Results, Stories | The Bridge" name="keywords"/>
  <meta content="en" name="language"/>
  <meta content="notranslate" name="google"/>
  <meta content="thebridge" name="author"/>
  <meta content="The Bridge - Home of Indian Sports" name="copyright"/>
  <meta content="follow, index" na

In [7]:
print(soup2.prettify())

<html class="b-reith-sans-font" lang="en-GB">
 <head>
  <script>
   window.Morph = {}
  </script>
  <script type="text/javascript">
   Morph.markPerformance = function(label) { if(window.performance && window.performance.mark) { window.performance.mark(label); } }; Morph.versions = {}; Morph.modules = {}; Morph.config = { cdn: "m.files.bbci.co.uk", pushEnv: "live" }; Morph.styles = {}; Morph.styleSrc = []; Morph.toInit = {bundles: [], registers: [], payloads: []};   Morph.initStyles = function() { if (!Morph.enhanced) { var eles = Array.prototype.slice.call(document.getElementsByTagName('noscript')); for (var i = 0, len = eles.length; i<len; i++) { if (eles[i].className === 'morph-core-style') { document.write(eles[i].innerText); eles[i].parentNode.removeChild(eles[i]); } } return; } for (var j = 0, jLen = Morph.styleSrc.length; j<jLen; j++) { document.write(Morph.styleSrc[j]); } Morph.styleSrc = []; };
  </script>
  <script type="text/javascript">
   Morph.forceCore = window.location.

In [8]:
print(soup3.prettify())

<!DOCTYPE html>
<html lang="en" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,minimum-scale=1,initial-scale=1" name="viewport"/>
  <!-- HTML5 shim and Respond.js IE8 support of HTML5 elements and media queries -->
  <!--[if lt IE 9]>
      <script src="https://oss.maxcdn.com/libs/html5shiv/3.7.0/html5shiv.js"></script>
      <script src="https://oss.maxcdn.com/libs/respond.js/1.4.2/respond.min.js"></script>
<![endif]-->
  <title>
   Sports News, Latest Sports News Headlines, Cricket News Today, World Cup 2019, Football News, Tennis News, Live Scores |  The Indian Express
  </title>
  <meta content="Sports News - Read Latest Sports News Today Headlines on IndianExpress.com. Find latest cricket news, tennis, football, hockey, World cup 2019, IPL 2019 Live Score Updates. Stay updated on Sports News" name="description">
   <meta content="sports news, latest news on sports, cricket news, live scores, football news, tennis news, latest fo

In [9]:
print(soup4.prettify())

﻿
<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Sports News: Latest Sports News, Live Scores, Results Today's Sports Headlines Updates
  </title>
  <meta charset="utf-8"/>
  <meta content="Sports News - Get Latest Sports News Today Headlines on Times Now news.com. Find latest cricket news, tennis, football, hockey, World cup 2021, IPL 2021, Live Score Updates. Stay updated on Sports News at Times Now" name="description"/>
  <meta content="TimesNow" name="author"/>
  <meta content="Sports News: Latest Sports News, Live Scores, Results Today's Sports Headlines Updates" itemprop="name"/>
  <meta content="Sports News - Get Latest Sports News Today Headlines on Times Now news.com. Find latest cricket news, tennis, football, hockey, World cup 2021, IPL 2021, Live Score Updates. Stay updated on Sports News at Times Now" itemprop="description"/>
  <meta content="https://iks.timesnownews.com/s/v1/img/logo/c_tnlogo.png" property="image"/>
  <meta content="https://iks.timesnownews.com/s/v

In [10]:
h1 = soup1.findAll('h4',class_='bd-heading-text')
h2 = soup2.findAll('a',class_='gs-c-promo-heading gs-o-faux-block-link__overlay-link sp-o-link-split__anchor gel-pica-bold')
h3 = soup3.findAll('h2',class_='title')
h4 = soup4.findAll('div',class_='text')

In [11]:
headlines1 = []
for i in range(0,len(h1)):
    headlines1.append(h1[i].get_text())
headlines1

['Suzy Hamilton — The story of an Olympic runner who resorted to prostitution',
 '"They asked when I\'ll become a mother, then asked whether I\'ll be able to play any more" — Sania Mirza',
 "The inspiring 'rags-to-riches' stories of 10 Indian Olympians",
 'Daughter of a construction labourer, sprinter VK Vismaya hopes for a good show by Indian relay team at Tokyo Olympics',
 "Here's what Usain Bolt predicts about Tokyo Olympics",
 'Better than Usain Bolt? Allyson Felix has a chance of becoming the greatest ever in Tokyo',
 'PV Sindhu to go for gold at Tokyo Olympics - How much possible is it?',
 'Tai Tzu will be the No.1 rival of PV Sindhu in Tokyo: Coach Park Tae-sang',
 'Pandemic did not impact my Olympic preparation: PV Sindhu',
 'We thought of finals, current shooters think of medals — Gagan Narang',
 "India's medal prospects at the Tokyo Olympics in shooting",
 'Indian shooters to train for Olympics in Tokyo without undergoing any quarantine',
 "Will quit if Vinesh Phogat doesn't 

In [12]:
headlines2 = []
for i in range(0,len(h2)):
    headlines2.append(h2[i].get_text())
headlines2

["Verstappen brands Hamilton 'disrespectful' after British GP crash",
 'England beat Pakistan by 45 runs to set up T20 series decider',
 'Cavendish denied record 35th Tour stage win as Pogacar wins second yellow jersey',
 'Rashford undecided on shoulder surgery - Solskjaer',
 'BBC Panorama: Thousands of racehorses killed in slaughterhouses',
 'Hamilton fights back from penalty to claim dramatic British GP win',
 'First athletes in Olympic village test positive for Covid & Team GB have eight isolating',
 'Atlanta sack ex-Man Utd defender Heinze as manager after 13 games ',
 "Liverpool and Man Utd in the hunt for Saul - Monday's gossip",
 'Broady set for debut as Gauff withdraws',
 'Crystal Palace sign England U21 defender Guehi from Chelsea',
 'Bucks one game from winning NBA title after beating Suns',
 "'It's about getting the ball to our danger players' - Vieira's impact at Palace",
 "'My daughter made me fall back in love with cycling after horrific Rio experience'",
 "Fit-again Jone

In [13]:
headlines3 = []
for i in range(0,len(h3)):
    headlines3.append(h3[i].get_text())
headlines3

['\nIshan Kishan, Prithvi Shaw lead Sri Lanka rout\n',
 '\nTiff over face mask exposes international betting racket\n',
 '\nTick, tick, tick… boom\n',
 '\nSpinners shine as England beat Pakistan to level T20 series 1-1\n',
 '\nPrithvi Shaw, Ishan Kishan finished the game in first 15 overs only: Shikhar Dhawan\n',
 '\nTadej Pogacar wins 2nd straight Tour de France\n',
 '\nFirst ODI: Birthday boy Ishan Kishan dazzles on debut, India beat Sri Lanka by 7 wickets\n',
 '\nWatch: Brendan Taylor’s bizarre hit-wicket as Bangladesh win ODI series against Zimbabwe\n',
 '\nWatch: Hamilton wins British GP, penalised for Verstappen’s race-ending crash\n',
 '\nMessi’s Copa America trophy picture becomes most-liked Instagram post by an athlete\n',
 '\nFrench Open champ Barbora Krejcikova triumphs at Prague Open\n',
 '\nAditi Ashok finishes career-best third with Thai partner in Great Lakes\n',
 '\n‘Welcome to the family’: Rahul Dravid to debutants Ishan Kishan, Suryakumar Yadav\n',
 '\nWatch: Joe Root

In [14]:
headlines4 = []
for i in range(0,len(h4)):
    headlines4.append(h4[i].get_text())
headlines4

['BOT vs PF, ECS T10 Sweden Dream11 Prediction Today: Fantasy cricket tips for Botkyrka vs Pakistanska Forening     ',
 'Tokyo Olympics: PV Sindhu, Sharath Kamal begin training ahead of Games      ',
 "I would've sacked team management and Babar: Akhtar slams Pakistan for bowling first in 2nd T20I vs England     ",
 'WWE Money in the Bank results: Big E wins MITB Ladder Match; Roman Reigns defeats Edge in main event     ',
 'Coco Gauff tests positive for COVID-19, to miss Tokyo Games     ',
 '[VIDEO] When Rahul Dravid played match-winning knock to eliminate Sri Lanka from 2002 NatWest series     ',
 "First competitors in athletes' village infected with COVID-19     ",
 'Tokyo Olympics: 8 Team GB members enter isolation after contact with COVID infected person     ',
 'South Africa unrest taking a toll of Proteas in Ireland: Mark Boucher      ',
 "British Grand Prix: Max Verstappen accuses Lewis Hamilton of 'disrespectful and unsportsmanlike behaviour'     ",
 "'Baby Cannibal' Tadej Pog

In [15]:
cluster_name1 = []
for i in range(0,len(h1)):
    cluster_name1.append('Sports')
cluster_name1

['Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports']

In [16]:
cluster_name2 = []
for i in range(0,len(h2)):
    cluster_name2.append('Sports')
cluster_name2

['Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports']

In [17]:
cluster_name3 = []
for i in range(0,len(h3)):
    cluster_name3.append('Sports')
cluster_name3

['Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports']

In [18]:
cluster_name4 = []
for i in range(0,len(h4)):
    cluster_name4.append('Sports')
cluster_name4

['Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports',
 'Sports']

In [19]:
source_1 = []
for i in range(0,len(h1)):
    source_1.append('The Bridge')
source_1

['The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge',
 'The Bridge']

In [20]:
source_2 = []
for i in range(0,len(h2)):
    source_2.append('BBC')
source_2

['BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC',
 'BBC']

In [21]:
source_3 = []
for i in range(0,len(h3)):
    source_3.append('Indian Express')
source_3

['Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express',
 'Indian Express']

In [22]:
source_4 = []
for i in range(0,len(h4)):
    source_4.append('Timesnow News')
source_4

['Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News',
 'Timesnow News']

In [23]:
news_1 = []
for i in range(0,len(h1)):
    news_1.append('India')
news_1

['India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India']

In [24]:
news_2 = []
for i in range(0,len(h2)):
    news_2.append('India')
news_2

['India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India',
 'India']

In [25]:
news_3 = []
for i in range(0,len(h3)):
    news_3.append('Global')
news_3

['Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global']

In [26]:
news_4 = []
for i in range(0,len(h4)):
    news_4.append('Global')
news_4

['Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global',
 'Global']

In [27]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()

In [28]:
df1['Headlines'] = headlines1
df1['Cluster'] = cluster_name1
df1['Source'] = source_1
df1['News Type'] = news_1

In [29]:
df1

,Headlines,Cluster,Source,News Type
0,Suzy Hamilton — The story of an Olympic runner...,Sports,The Bridge,India
1,"""They asked when I'll become a mother, then as...",Sports,The Bridge,India
2,The inspiring 'rags-to-riches' stories of 10 I...,Sports,The Bridge,India
3,"Daughter of a construction labourer, sprinter ...",Sports,The Bridge,India
4,Here's what Usain Bolt predicts about Tokyo Ol...,Sports,The Bridge,India
5,Better than Usain Bolt? Allyson Felix has a ch...,Sports,The Bridge,India
6,PV Sindhu to go for gold at Tokyo Olympics - H...,Sports,The Bridge,India
7,Tai Tzu will be the No.1 rival of PV Sindhu in...,Sports,The Bridge,India
8,Pandemic did not impact my Olympic preparation...,Sports,The Bridge,India
9,"We thought of finals, current shooters think o...",Sports,The Bridge,India


In [30]:
df2['Headlines'] = headlines2
df2['Cluster'] = cluster_name2
df2['Source'] = source_2
df2['News Type'] = news_2

In [31]:
df2

,Headlines,Cluster,Source,News Type
0,Verstappen brands Hamilton 'disrespectful' aft...,Sports,BBC,India
1,England beat Pakistan by 45 runs to set up T20...,Sports,BBC,India
2,Cavendish denied record 35th Tour stage win as...,Sports,BBC,India
3,Rashford undecided on shoulder surgery - Solsk...,Sports,BBC,India
4,BBC Panorama: Thousands of racehorses killed i...,Sports,BBC,India
...,...,...,...,...
58,Sportshour podcast: The last woman at Silverstone,Sports,BBC,India
59,World Football: The Olympics are coming,Sports,BBC,India
60,"Stumped podcast: Michael Holding on racism, so...",Sports,BBC,India
61,Tailenders podcast: The Thousand,Sports,BBC,India


In [32]:
df3['Headlines'] = headlines3
df3['Cluster'] = cluster_name3
df3['Source'] = source_3
df3['News Type'] = news_3

In [33]:
df3

,Headlines,Cluster,Source,News Type
0,"\nIshan Kishan, Prithvi Shaw lead Sri Lanka ro...",Sports,Indian Express,Global
1,\nTiff over face mask exposes international be...,Sports,Indian Express,Global
2,"\nTick, tick, tick… boom\n",Sports,Indian Express,Global
3,\nSpinners shine as England beat Pakistan to l...,Sports,Indian Express,Global
4,"\nPrithvi Shaw, Ishan Kishan finished the game...",Sports,Indian Express,Global
5,\nTadej Pogacar wins 2nd straight Tour de Fran...,Sports,Indian Express,Global
6,\nFirst ODI: Birthday boy Ishan Kishan dazzles...,Sports,Indian Express,Global
7,\nWatch: Brendan Taylor’s bizarre hit-wicket a...,Sports,Indian Express,Global
8,"\nWatch: Hamilton wins British GP, penalised f...",Sports,Indian Express,Global
9,\nMessi’s Copa America trophy picture becomes ...,Sports,Indian Express,Global


In [34]:
df4['Headlines'] = headlines4
df4['Cluster'] = cluster_name4
df4['Source'] = source_4
df4['News Type'] = news_4

In [35]:
df4

,Headlines,Cluster,Source,News Type
0,"BOT vs PF, ECS T10 Sweden Dream11 Prediction T...",Sports,Timesnow News,Global
1,"Tokyo Olympics: PV Sindhu, Sharath Kamal begin...",Sports,Timesnow News,Global
2,I would've sacked team management and Babar: A...,Sports,Timesnow News,Global
3,WWE Money in the Bank results: Big E wins MITB...,Sports,Timesnow News,Global
4,"Coco Gauff tests positive for COVID-19, to mis...",Sports,Timesnow News,Global
5,[VIDEO] When Rahul Dravid played match-winning...,Sports,Timesnow News,Global
6,First competitors in athletes' village infecte...,Sports,Timesnow News,Global
7,Tokyo Olympics: 8 Team GB members enter isolat...,Sports,Timesnow News,Global
8,South Africa unrest taking a toll of Proteas i...,Sports,Timesnow News,Global
9,British Grand Prix: Max Verstappen accuses Lew...,Sports,Timesnow News,Global


In [36]:
frames = [df1,df2,df3,df4]

In [37]:
result = pd.concat(frames,join='inner')
result 

,Headlines,Cluster,Source,News Type
0,Suzy Hamilton — The story of an Olympic runner...,Sports,The Bridge,India
1,"""They asked when I'll become a mother, then as...",Sports,The Bridge,India
2,The inspiring 'rags-to-riches' stories of 10 I...,Sports,The Bridge,India
3,"Daughter of a construction labourer, sprinter ...",Sports,The Bridge,India
4,Here's what Usain Bolt predicts about Tokyo Ol...,Sports,The Bridge,India
...,...,...,...,...
30,\n\r\n Karnataka Class 10 ...,Sports,Timesnow News,Global
31,\n\r\n Editor’s Take | HDF...,Sports,Timesnow News,Global
32,\n\r\n Data of 40 Indian j...,Sports,Timesnow News,Global
33,\n\r\n Over 40 Indian jour...,Sports,Timesnow News,Global


In [38]:
result = result.reset_index()

In [39]:
result = result.drop('index',axis=1)
result

,Headlines,Cluster,Source,News Type
0,Suzy Hamilton — The story of an Olympic runner...,Sports,The Bridge,India
1,"""They asked when I'll become a mother, then as...",Sports,The Bridge,India
2,The inspiring 'rags-to-riches' stories of 10 I...,Sports,The Bridge,India
3,"Daughter of a construction labourer, sprinter ...",Sports,The Bridge,India
4,Here's what Usain Bolt predicts about Tokyo Ol...,Sports,The Bridge,India
...,...,...,...,...
166,\n\r\n Karnataka Class 10 ...,Sports,Timesnow News,Global
167,\n\r\n Editor’s Take | HDF...,Sports,Timesnow News,Global
168,\n\r\n Data of 40 Indian j...,Sports,Timesnow News,Global
169,\n\r\n Over 40 Indian jour...,Sports,Timesnow News,Global


In [40]:
result.to_csv('Sports_News_Cluster.csv')